## Test implementation
----

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from CBFW import CBFW
from MDLPC import MDLP_Discretizer as mdlp

import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from data_utils import datasets

In [3]:
# read dataset
train = pd.read_csv('./data/monks-1.train', header=None, sep=' ')
X_train, y_train = train.iloc[:,2:-1], train.iloc[:,1]

test  = pd.read_csv('./data/monks-1.test', header=None, sep=' ')
X_test, y_test = test.iloc[:,2:-1], test.iloc[:,1]

In [4]:
def k_fold_train(dataset, K=10):
    X, y, mdlpc_cols = dataset()
    
    skf = StratifiedKFold(n_splits=K)
    skf.get_n_splits(X, y)
    
    accs = []
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # quantize x values using MDLP algorithm
        if len(mdlpc_cols)>0:
            discretizer = mdlp(features=mdlpc_cols)
            discretizer.fit(X_train, y_train)
            X_train = discretizer.transform(X_train)
            X_test  = discretizer.transform(X_test)
        
        model = CBFW()
        model.fit(X_train, y_train)
        
        y_pred = model.predict(X_test)
        accs.append(accuracy_score(y_test, y_pred))
        
    return np.mean(accs), np.std(accs)

In [5]:
k_fold_train(datasets['waveform-5000'])

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1mUntyped global name 'entropy_numpy':[0m [1m[1mCannot determine Numba type of <class 'function'>[0m
[1m
File "MDLPC\Entropy.py", line 40:[0m
[1mdef cut_point_information_gain_numpy(X, y, cut_point):
    <source elided>
    '''
[1m    entropy_full = entropy_numpy(y)  # compute entropy of full dataset (w/o split)
[0m    [1m^[0m[0m
[0m